In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Data/Split_highway.csv')

In [3]:
df.head()

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,id,version,tags,osm_type,fid,id_2,left,top,right,bottom
0,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Macondray Lane,NaN,...,5004035,-1,NaN,way,9789,9789,-122.417536,37.799515,-122.416396,37.798612
1,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11603,11603,-122.400434,37.719094,-122.399294,37.718191
2,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11718,11718,-122.399294,37.719094,-122.398154,37.718191
3,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11719,11719,-122.399294,37.718191,-122.398154,37.717287
4,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11720,11720,-122.399294,37.717287,-122.398154,37.716383


In [4]:
df.columns

Index(['bicycle', 'bridge', 'cycleway', 'footway', 'highway', 'lanes', 'lit',
       'maxspeed', 'name', 'oneway', 'ref', 'service', 'sidewalk', 'surface',
       'id', 'version', 'tags', 'osm_type', 'fid', 'id_2', 'left', 'top',
       'right', 'bottom'],
      dtype='object')

In [5]:
df = df.drop(['version', 'id_2'], axis=1)

In [6]:
df['tags'].value_counts()

{"crossing":"marked"}                                                                                                                                                                                                                                             2126
{"layer":"-1"}                                                                                                                                                                                                                                                     493
{"tiger:cfcc":"A41","tiger:county":"San Francisco, CA","tiger:name_type":"St"}                                                                                                                                                                                     442
{"crossing":"unmarked"}                                                                                                                                                                                            

In [7]:
groups = df.groupby('fid')

In [8]:
tile_value_count_df = df['fid'].value_counts().sort_values().to_frame().reset_index().rename(columns={'index': 'tile_id','fid':'count'})

In [23]:
tile_value_count_df[tile_value_count_df['count']==1]['tile_id']

0        2898
1       12182
2       14240
3       14355
4       14470
        ...  
1337     2712
1338    13681
1339    11478
1340     2714
1341     4383
Name: tile_id, Length: 1342, dtype: int64

In [24]:
temp_tile_count_one = df[df['fid'].isin(tile_value_count_df[tile_value_count_df['count']==1]['tile_id'])]

In [25]:
temp_tile_count_one

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,sidewalk,surface,id,tags,osm_type,fid,left,top,right,bottom
37,NaN,NaN,NaN,NaN,residential,2.0,NaN,NaN,12th Avenue,NaN,...,both,NaN,8914970,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4670,-122.468841,37.748913,-122.467701,37.748010
46,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,12th Avenue,NaN,...,both,asphalt,8914974,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4537,-122.469981,37.765178,-122.468841,37.764275
48,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,12th Avenue,NaN,...,both,asphalt,8914974,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4539,-122.469981,37.763371,-122.468841,37.762467
73,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Arelius Walker Drive,NaN,...,NaN,NaN,8914996,"{""old_name"":""Fitch Street"",""tiger:cfcc"":""A41"",...",way,14348,-122.373072,37.732648,-122.371932,37.731745
91,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Aquavista Way,NaN,...,NaN,NaN,8915002,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,6620,-122.449459,37.753431,-122.448319,37.752528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83040,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4522,-122.469981,37.778732,-122.468841,37.777829
83042,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4524,-122.469981,37.776925,-122.468841,37.776021
83045,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4527,-122.469981,37.774214,-122.468841,37.773311
83167,NaN,NaN,NaN,NaN,residential,2.0,NaN,25 mph,45th Avenue,no,...,both,asphalt,1117521594,"{""cycleway:both"":""no"",""parking:lane:left"":""per...",way,844,-122.506465,37.776925,-122.505325,37.776021


In [42]:
temp_tile_count_one.columns
for col in temp_tile_count_one.columns:
    print(f"{col}: {temp_tile_count_one[col].isnull().values.all()}, \
    {temp_tile_count_one[col].isnull().mean() * 100}")


bicycle: False,     92.10134128166915
bridge: False,     99.2548435171386
cycleway: False,     95.38002980625932
footway: False,     99.55290611028316
highway: False,     0.0
lanes: False,     83.90461997019374
lit: False,     62.74217585692996
maxspeed: False,     87.77943368107303
name: False,     21.982116244411326
oneway: False,     87.10879284649776
ref: False,     99.47839046199702
service: False,     99.2548435171386
sidewalk: False,     50.52160953800298
surface: False,     51.41579731743666
id: False,     0.0
tags: False,     16.98956780923994
osm_type: False,     0.0
fid: False,     0.0
left: False,     0.0
top: False,     0.0
right: False,     0.0
bottom: False,     0.0
